Chuẩn Bị Dữ Liệu

Giả sử chúng ta có các cặp câu và nhãn tương ứng:

In [ ]:
import torch

queries = ["PhoBERT là một mô hình mạnh mẽ.", "Học sâu là tương lai của AI."]
documents = ["PhoBERT được phát triển bởi VinAI.", "Học sâu cần nhiều dữ liệu và tính toán."]
labels = [1, 0]  # 1: tương đồng, 0: không tương đồng


Tạo Embeddings Bằng PhoBERT

In [ ]:
from transformers import AutoModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
model = AutoModel.from_pretrained("vinai/phobert-base")

def get_embeddings(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :]  # Sử dụng embedding của token [CLS]

query_embeddings = get_embeddings(queries)
document_embeddings = get_embeddings(documents)


Tính Toán Sự Tương Đồng

Có nhiều cách để tính toán sự tương đồng giữa hai vectors. Một cách phổ biến là sử dụng cosine similarity:

In [ ]:
import torch.nn.functional as F

similarities = F.cosine_similarity(query_embeddings, document_embeddings)
print(similarities)


Sử Dụng Lớp Phân Loại và CrossEntropyLoss

Nếu bạn muốn sử dụng lớp phân loại để dự đoán nhãn của cặp câu, bạn có thể làm như sau:

In [ ]:
import torch.nn as nn

class SimilarityClassifier(nn.Module):
    def __init__(self):
        super(SimilarityClassifier, self).__init__()
        self.phobert = AutoModel.from_pretrained("vinai/phobert-base")
        self.classifier = nn.Linear(768, 2)  # 2 lớp: tương đồng và không tương đồng

    def forward(self, input_ids, attention_mask):
        outputs = self.phobert(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = outputs.last_hidden_state[:, 0, :]  # Sử dụng embedding của token [CLS]
        logits = self.classifier(cls_output)
        return logits

# Chuẩn bị dữ liệu
inputs = tokenizer(queries + documents, padding=True, truncation=True, return_tensors="pt")
labels = torch.tensor(labels)

# Tạo mô hình và hàm mất mát
model = SimilarityClassifier()
criterion = nn.CrossEntropyLoss()

# Dự đoán và tính toán mất mát
logits = model(inputs['input_ids'], inputs['attention_mask'])
loss = criterion(logits, labels)
print(f"Loss: {loss.item()}")
